In [232]:
import requests
import json
import pandas as pd
from datetime import datetime
import sqlite3
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [26]:
"""
Según la documentación de Socrata Open Data API cada conjunto de datos se representa por un identificador único
de ocho caracteres alfanuméricos divididos en dos frases de cuatro caracteres cada una, separadas por un guion.

La url + el unique id y sin la extensión .json nos lleva a una página web con la descripción de los datos
"""

'\nSegún la documentación de Socrata Open Data API cada conjunto de datos se representa por un identificador único\nde ocho caracteres alfanuméricos divididos en dos frases de cuatro caracteres cada una, separadas por un guion.\nLa url + el unique id y sin la extensión .json nos lleva a una página web con la descripción de los datos\n'

In [152]:
resource = "https://datos.gov.co/resource/"
parameter = "7cci-nqqb.json"
url = resource + parameter

# Realiza la solicitud GET al enlace y guarda la respuesta en la variable data, luego lo convertimos en un dataframe
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
df = df.infer_objects()

In [92]:
df.nunique()

orden                      1000
fecha                        85
a_o                           1
mes                           3
d_a                           7
gravedad                      3
peaton                        3
automovil                     4
campaero                      3
camioneta                     3
micro                         2
buseta                        3
bus                           2
camion                        3
volqueta                      2
moto                          4
bicicleta                     2
otro                          3
via_1                        26
barrio                      116
hora                        269
entidad                       1
nombrecomuna                 21
propietario_de_veh_culo       2
diurnio_nocturno              2
hora_restriccion_moto         3
dtype: int64

In [153]:
# Se elimina la columna orden ya que nuestro dataframe tiene un indice para identifcar las filas
# Se elimina las columnas año, mes y día porque ya se encuentran implicitamente en la columan fecha
df2 = df.drop(['orden','a_o','mes','d_a'], axis=1)

In [154]:
df2['fecha'] = pd.to_datetime(df2['fecha'],format="%d/%m/%Y")
df2['peaton'] = df2['peaton'].astype(int)
df2['automovil'] = df2['automovil'].astype(int)
df2['campaero'] = df2['campaero'].astype(int)
df2['camioneta'] = df2['camioneta'].astype(int)
df2['micro'] = df2['micro'].astype(int)
df2['buseta'] = df2['buseta'].astype(int)
df2['bus'] = df2['bus'].astype(int)
df2['camion'] = df2['camion'].astype(int)
df2['volqueta'] = df2['volqueta'].astype(int)
df2['moto'] = df2['moto'].astype(int)
df2['bicicleta'] = df2['bicicleta'].astype(int)
df2['otro'] = df2['otro'].astype(int)
df2['gravedad'] = df2['gravedad'].astype('category')
df2['entidad'] = df2['entidad'].astype('category')
df2['propietario_de_veh_culo'] = df2['propietario_de_veh_culo'].astype('category')
df2['diurnio_nocturno'] = (df2['diurnio_nocturno'] == 'Diurno').astype(bool)
df2.rename(columns={'diurnio_nocturno': 'es_diurno'}, inplace=True)
df2['hora_restriccion_moto'] = df2['hora_restriccion_moto'].astype('category')

In [155]:
# Realizamos una transformación en la columna hora para que haga match con los formatos de hora
new_time = []
format = '%I:%M:%S %p'
for time in df['hora']:
    time2 = time
    time3 = time
    time2 = time.replace('p.\xa0m.', 'pm')
    time3 = time2.replace('a.\xa0m.', 'am')
    time2 = time3
    time2 = datetime.strptime(time2,format).time()
    new_time.append(time2)
df2['hora'] = new_time    

In [229]:
df2.dtypes

fecha                      datetime64[ns]
gravedad                         category
peaton                              int32
automovil                           int32
campaero                            int32
camioneta                           int32
micro                               int32
buseta                              int32
bus                                 int32
camion                              int32
volqueta                            int32
moto                                int32
bicicleta                           int32
otro                                int32
via_1                              object
barrio                             object
hora                               object
entidad                          category
nombrecomuna                       object
propietario_de_veh_culo          category
es_diurno                            bool
hora_restriccion_moto            category
dtype: object

In [222]:
# Creamos las subtablas y una columna que identifique los campos unicos
df_vehiculo = pd.DataFrame()
df_lugar = pd.DataFrame()
df_fecha = pd.DataFrame()
df_hora = pd.DataFrame()
vehiculos = ['peaton','automovil','campaero','camioneta','micro','buseta','bus','camion','volqueta','moto','bicicleta','otro']
df_vehiculo = df2.loc[:, vehiculos]
df_vehiculo.insert(0,'id_vehiculo', pd.factorize(df_vehiculo.apply(tuple, axis=1))[0])
lugar = ['nombrecomuna','barrio','via_1']
df_lugar = df2.loc[:,lugar]
df_lugar.insert(0,'id_lugar', pd.factorize(df_lugar.apply(tuple, axis=1))[0])
df_fecha['fecha'] = df2.loc[:,'fecha']
df_fecha.insert(0,'id_fecha', pd.factorize(df_fecha.apply(tuple, axis=1))[0])
df_hora['hora'] = df2.loc[:, 'hora']
df_hora.insert(0,'id_hora', pd.factorize(df_hora.apply(tuple, axis=1))[0])
df_fecha

,id_fecha,fecha
0,0,2012-01-01
1,0,2012-01-01
2,0,2012-01-01
3,0,2012-01-01
4,0,2012-01-01
...,...,...
995,83,2012-03-24
996,83,2012-03-24
997,84,2012-03-25
998,84,2012-03-25


In [221]:
# Ahora creamos nuestro dataframe principal y eliminado los campos que ya están en nuestras subtablas
df_incidente = pd.DataFrame()
redundantes = vehiculos + lugar + ['fecha','hora']
df_incidente = df2.drop(redundantes, axis=1)
df_incidente = pd.merge(df_lugar[['id_lugar']], df_incidente, left_index=True, right_index=True)
df_incidente = pd.merge(df_vehiculo[['id_vehiculo']], df_incidente, left_index=True, right_index=True)
df_incidente = pd.merge(df_hora[['id_hora']], df_incidente, left_index=True, right_index=True)
df_incidente = pd.merge(df_fecha[['id_fecha']], df_incidente, left_index=True, right_index=True)
df_incidente

,id_fecha,id_hora,id_vehiculo,id_lugar,gravedad,entidad,propietario_de_veh_culo,es_diurno,hora_restriccion_moto
0,0,0,0,0,Con Heridos,AGENTES DTB,Particular,True,No aplica
1,0,1,1,1,Solo Daños,AGENTES DTB,Empresa,True,No aplica
2,0,2,2,2,Solo Daños,AGENTES DTB,Particular,True,No aplica
3,0,3,1,3,Solo Daños,AGENTES DTB,Particular,False,No aplica
4,0,4,3,4,Con Heridos,AGENTES DTB,Particular,False,En restricción
...,...,...,...,...,...,...,...,...,...
995,83,69,27,246,Solo Daños,AGENTES DTB,Empresa,True,No aplica
996,83,182,4,145,Con Heridos,AGENTES DTB,Particular,True,Sin restricción
997,84,268,10,17,Con Heridos,AGENTES DTB,Particular,False,En restricción
998,84,126,16,144,Con Heridos,AGENTES DTB,Particular,True,Sin restricción


In [227]:
# Ahora limpiamos los duplicados de las subtablas
df_fecha = df_fecha.drop_duplicates()
df_hora = df_hora.drop_duplicates()
df_lugar = df_lugar.drop_duplicates()
df_vehiculo = df_vehiculo.drop_duplicates()
df_vehiculo

,id_vehiculo,peaton,automovil,campaero,camioneta,micro,buseta,bus,camion,volqueta,moto,bicicleta,otro
0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,0,0,0,0,0,0
4,3,1,0,0,0,0,0,0,0,0,1,0,0
5,4,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,92,0,3,0,0,0,0,0,0,0,1,0,0
912,93,1,0,0,0,0,0,0,0,0,3,0,0
925,94,0,0,0,1,0,0,0,0,0,0,1,0
927,95,0,1,0,0,0,0,0,0,0,1,1,0


In [233]:
# Creamos nuestra base de datos y agregamos nuestras tablas
conn = sqlite3.connect('basedatos.db')
df_incidente.to_sql('incidentes', conn, index=False, if_exists='replace')
df_fecha.to_sql('fechas', conn, index=False, if_exists='replace')
df_hora.to_sql('horas', conn, index=False, if_exists='replace')
df_vehiculo.to_sql('vehiculos', conn, index=False, if_exists='replace')
df_lugar.to_sql('lugares', conn, index=False, if_exists='replace')
conn.close()

In [235]:
# Nos conectamos a la base de datos y podemos guardar el resultado de las querys
conn = sqlite3.connect('basedatos.db')
query = "SELECT * FROM incidentes"
df_resultado = pd.read_sql_query(query, conn)
df_resultado

,id_fecha,id_hora,id_vehiculo,id_lugar,gravedad,entidad,propietario_de_veh_culo,es_diurno,hora_restriccion_moto
0,0,0,0,0,Con Heridos,AGENTES DTB,Particular,1,No aplica
1,0,1,1,1,Solo Daños,AGENTES DTB,Empresa,1,No aplica
2,0,2,2,2,Solo Daños,AGENTES DTB,Particular,1,No aplica
3,0,3,1,3,Solo Daños,AGENTES DTB,Particular,0,No aplica
4,0,4,3,4,Con Heridos,AGENTES DTB,Particular,0,En restricción
...,...,...,...,...,...,...,...,...,...
995,83,69,27,246,Solo Daños,AGENTES DTB,Empresa,1,No aplica
996,83,182,4,145,Con Heridos,AGENTES DTB,Particular,1,Sin restricción
997,84,268,10,17,Con Heridos,AGENTES DTB,Particular,0,En restricción
998,84,126,16,144,Con Heridos,AGENTES DTB,Particular,1,Sin restricción
